In [1]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine, inspect
import io
import json
import os
import re

In [2]:
logs_vols = pd.read_csv(r'http://sc-e.fr/docs/logs_vols_2024-06-18.csv')
logs_vols.head(10)

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V03826762,A380_4294,2024-06-18,6.4,"{'temp': '5.1°C', 'pressure': '905.1 hPa', 'vi...",0
1,V09631780,CRJ700_3181,2024-06-18,8.7,"{'temp': '-2.7°C', 'pressure': '933.6 hPa', 'v...",0
2,V08439611,A330_0921,2024-06-18,4.7,"{'temp': '11.4°C', 'pressure': '889.4 hPa', 'v...",0
3,V00425601,B747_6800,2024-06-18,2.6,"{'temp': '-8.9°C', 'pressure': '1072.6 hPa', '...",0
4,V03836886,A380_2079,2024-06-18,3.2,"{'temp': '-6.5°C', 'pressure': '872.3 hPa', 'v...",0
5,V08406955,B737_1117,2024-06-18,7.6,"{'temp': '9.5°C', 'pressure': '943.6 hPa', 'vi...",0
6,V08050316,A320_2562,2024-06-18,5.0,"{'temp': '-13.2°C', 'pressure': '1047.5 hPa', ...",1
7,V09424063,B757_0697,2024-06-18,5.4,"{'temp': '-9.4°C', 'pressure': '970.7 hPa', 'v...",0
8,V09946874,A350_3976,2024-06-18,8.2,"{'temp': '10.9°C', 'pressure': '933.0 hPa', 'v...",0
9,V00954197,A321_1968,2024-06-18,2.6,"{'temp': '1.8°C', 'pressure': '971.1 hPa', 'vi...",0


In [3]:
logs_vols.shape

(84, 6)

In [4]:
aero_e170_5551 = logs_vols[logs_vols['aero_linked'] == 'E170_5551']
aero_e170_5551

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant


In [5]:
# date du jour
date_today = datetime.today().strftime('%Y-%m-%d')

In [6]:
# Configurer la connexion à la base de données
db_path = r'C:\Document\Python\Projet\Projet_3\Top_gun\bdd_sql\top_gun_data_1.db'
engine = create_engine(f'sqlite:///{db_path}')

date_defini = '2024-06-21'

In [7]:
# Fonction pour nettoyer les données
def clean_data(df):
    # Fonction pour corriger le format JSON
    def fix_json_format(x):
        return x.replace("'", '"')
    
    # Transformer la colonne 'sensor_data' de JSON en plusieurs colonnes
    df['sensor_data'] = df['sensor_data'].apply(lambda x: json.loads(fix_json_format(x)))
    sensor_data_df = pd.json_normalize(df['sensor_data'])
    df = pd.concat([df.drop(columns=['sensor_data']), sensor_data_df], axis=1)
    
    # Transformer la colonne 'jour_vol' au format date
    df['jour_vol'] = pd.to_datetime(df['jour_vol'], format='%Y-%m-%d', errors='coerce').dt.date
    
    # Nettoyer les colonnes 'temp', 'pressure', et 'vibrations'
    df['temp en °C'] = df['temp'].replace('°C', '', regex=True).astype(float)
    df['pressure en hPa'] = df['pressure'].replace('hPa', '', regex=True).astype(float)
    df['vibrations en m/s²'] = df['vibrations'].replace('m/s²', '', regex=True).astype(float).round(3)
    df.drop(columns=['temp', 'pressure', 'vibrations'], inplace=True)
    
    # Exclure les lignes où 'aero_linked' est 'E170_5551'
    df = df[df['aero_linked'] != 'E170_5551']

    return df

In [8]:
# Vérifier si une table existe déjà dans la base de données
def table_exists(engine, table_name):
    inspector = inspect(engine)
    return inspector.has_table(table_name)

# Fonction pour initialiser la base de données avec le fichier d'origine si la table n'existe pas
def initialize_database(engine):
    if not table_exists(engine, 'logs_vols'):
        logs_vols_data = pd.read_csv(r'C:\Document\Python\Projet\Projet_3\Top_gun\Datasets\df_logs_vols\logs_vols_2024-06-02.csv')
        
        # Nettoyer les données
        cleaned_data = clean_data(logs_vols_data)
        
        # Stocker les données dans la base de données SQL
        cleaned_data.to_sql(name='logs_vols', con=engine, if_exists='replace', index=False)
        print("Base de données initialisée avec le fichier d'origine.")
    else:
        print("La table 'logs_vols' existe déjà, aucune initialisation nécessaire.")

# Fonction pour récupérer les données logs_vols et les stocker dans la base de données SQL
def recup_log_vols(date, engine):
    url = f"http://sc-e.fr/docs/logs_vols_{date}.csv"

    response = requests.get(url)
    data = pd.read_csv(url)

    # Nettoyer les données
    data = clean_data(data)

    # Stocker les données dans la base de données SQL
    data.to_sql(name='logs_vols', con=engine, if_exists='append', index=False)
    print(f"Données de {date} ajoutées à la base de données.")

# Appeler la fonction pour initialiser la base de données avec le fichier d'origine si nécessaire
initialize_database(engine)

# Appeler la fonction pour récupérer et stocker les logs de vols
recup_log_vols(date_defini, engine)

La table 'logs_vols' existe déjà, aucune initialisation nécessaire.
Données de 2024-06-21 ajoutées à la base de données.
